In [ ]:
# import modules
%matplotlib inline
import MDAnalysis as md
import MDAnalysis.analysis.distances
import numpy as np
import math as math
import pandas as pd
import matplotlib.pyplot as plt

# Calculating Predicted Intensity Ratios

In [ ]:
# defining constants
K = 1.23e-44 # m^6 s^-2
tau_C = 5e-9 # tauC for disordered RNC, 12ns, in seconds
larmor_H = 800.284e6*2*math.pi # s^-1, larmor frequency proton
#t = 0.0111 # total evolution time of the transverse proton magnetization during the NMR experiment, 11.1ms, in seconds
R2H = 100.0 # in s^-1
R2MQ = 100. # in s^-1
DELTA = 5.6e-3 # delay time in s

In [ ]:
# MD trajectory files, identical replicas of simulation with random initial velocities of atoms
print("Reading in trajectory ...")
universes = [
    md.Universe('./min.gro','./traj.xtc')
    ]

# read in amide positions -> (Nuniverses*Ntraj, Nresi, 3)
Ntraj = len(universes[0].trajectory) # number of frames per trajectory
Nuniv = len(universes) # number of replicas
u = md.Universe('./min.gro','./traj.xtc')

### Probe site c657

In [ ]:
# select calphas and remove probe calpha from selection to avoid generation of 0 values
print("Reading in trajectory for calpha positions...")
ag = u.select_atoms('resid 1:105 and not(resid 12)')
data = np.array([ag.ts.positions for ts in u.trajectory for u in universes])
#data.shape # OK (Nuniverses*Ntraj, Nresi, 3)


spinlabels = {
    'SC_657': 'resid 12' # glutamate
    }


rates_657 = []
ratios_657 = []

for spinlabel, selector in spinlabels.items():
    print("Reading in spin label positions for {}...".format(spinlabel))
    # read in spin label position across trajectory -> (Nuniv*Ntraj, 1, 3)
    u = md.Universe('./min.gro','./traj.xtc')
    ag = u.select_atoms(selector)
    dataSL = np.array([ag.ts.positions for ts in u.trajectory for u in universes])


    print("Calculating distances...")
    # distance between amides and spin labels -> (Nuniv*Ntraj, Nresi, 3)
    dx = data - dataSL
    
    # apply a little function to calculate r^-6 for each (dx, dy, dz)
    calc_rminus6 = lambda dx: (np.linalg.norm(dx)*1e-10)**-6
    rminus6 = np.apply_along_axis(calc_rminus6, 2, dx)
    
    # saving array of all distances (before averaging)
    rm6_filename = './saved_rm6_{}'.format(spinlabel)
    np.save(rm6_filename,rminus6)
   

    
    print("Averaging the ensemble and calculating PRE distances...")
    # finally take the ensemble average
    #rminus6_avg = np.mean(rminus6[0:10000,:],axis=0) # only average first 10000 steps
    rminus6_avg = np.mean(rminus6,axis=0)
    
    # calculating 'average' distances in Angstroms
    r_angstrom = (rminus6_avg**(-1/6))*1e10
    distance_filename = './saved_avg_dist_{}'.format(spinlabel)
    np.save(distance_filename,r_angstrom)
   

    
    print("Calculating PRE rates...")
    # finally take the ensemble average
    #rminus6_avg = np.mean(rminus6[0:10000,:],axis=0) # only average first 10000 steps
    #rminus6_avg = np.mean(rminus6,axis=0)
    rate = (K*rminus6_avg)*(4*tau_C+((3*tau_C)/(1+(larmor_H**2)*(tau_C**2))))
    rates_657 = rate
    rate_filename = "./saved_rate_{}".format(spinlabel)
    np.save(rate_filename,rates_657)
    
    print("Calculating intensity ratios...")
    ratios_657 = (R2H*np.exp(-2*DELTA*rate)/(R2H+rate))*(R2MQ/(R2MQ+rate))
    ratio_filename = './saved_ratio_{}'.format(spinlabel)
    np.save(ratio_filename,ratios_657)
 
                                                           
                                                           
print("Done!")

### Probe site c699

In [ ]:
# select calphas and remove probe calpha from selection to avoid generation of 0 values
print("Reading in trajectory for calpha positions...")
ag = u.select_atoms('resid 1:105 and not(resid 54)')
data = np.array([ag.ts.positions for ts in u.trajectory for u in universes])
#data.shape # OK (Nuniverses*Ntraj, Nresi, 3)


spinlabels = {
    'SC_699': 'resid 54' # aspartate 
    }


rates_699 = []
ratios_699 = []

for spinlabel, selector in spinlabels.items():
    print("Reading in spin label positions for {}...".format(spinlabel))
    # read in spin label position across trajectory -> (Nuniv*Ntraj, 1, 3)
    u = md.Universe('./min.gro','./traj.xtc')
    ag = u.select_atoms(selector)
    dataSL = np.array([ag.ts.positions for ts in u.trajectory for u in universes])


    print("Calculating distances...")
    # distance between amides and spin labels -> (Nuniv*Ntraj, Nresi, 3)
    dx = data - dataSL
    
    # apply a little function to calculate r^-6 for each (dx, dy, dz)
    calc_rminus6 = lambda dx: (np.linalg.norm(dx)*1e-10)**-6
    rminus6 = np.apply_along_axis(calc_rminus6, 2, dx)
    
    # saving array of all distances (before averaging)
    rm6_filename = './saved_rm6_{}'.format(spinlabel)
    np.save(rm6_filename,rminus6)
   

    
    print("Averaging the ensemble and calculating PRE distances...")
    # finally take the ensemble average
    #rminus6_avg = np.mean(rminus6[0:10000,:],axis=0) # only average first 10000 steps
    rminus6_avg = np.mean(rminus6,axis=0)
    
    # calculating 'average' distances in Angstroms
    r_angstrom = (rminus6_avg**(-1/6))*1e10
    distance_filename = './saved_avg_dist_{}'.format(spinlabel)
    np.save(distance_filename,r_angstrom)
   

    
    print("Calculating PRE rates...")
    # finally take the ensemble average
    #rminus6_avg = np.mean(rminus6[0:10000,:],axis=0) # only average first 10000 steps
    #rminus6_avg = np.mean(rminus6,axis=0)
    rate = (K*rminus6_avg)*(4*tau_C+((3*tau_C)/(1+(larmor_H**2)*(tau_C**2))))
    rates_699 = rate
    rate_filename = "./saved_rate_{}".format(spinlabel)
    np.save(rate_filename,rates_699)
    
    print("Calculating intensity ratios...")
    ratios_699 = (R2H*np.exp(-2*DELTA*rate)/(R2H+rate))*(R2MQ/(R2MQ+rate))
    ratio_filename = './saved_ratio_{}'.format(spinlabel)
    np.save(ratio_filename,ratios_699)
 
                                                           
                                                           
print("Done!") 

### Probe site c744

In [ ]:
# select calphas and remove probe calpha from selection to avoid generation of 0 values
print("Reading in trajectory for calpha positions...")
ag = u.select_atoms('resid 1:105 and not(resid 99)')
data = np.array([ag.ts.positions for ts in u.trajectory for u in universes])
#data.shape # OK (Nuniverses*Ntraj, Nresi, 3)


spinlabels = {
    'SC_744': 'resid 99' # aspartate
    }


rates_744 = []
ratios_744 = []

for spinlabel, selector in spinlabels.items():
    print("Reading in spin label positions for {}...".format(spinlabel))
    # read in spin label position across trajectory -> (Nuniv*Ntraj, 1, 3)
    u = md.Universe('./min.gro','./traj.xtc')
    ag = u.select_atoms(selector)
    dataSL = np.array([ag.ts.positions for ts in u.trajectory for u in universes])


    print("Calculating distances...")
    # distance between amides and spin labels -> (Nuniv*Ntraj, Nresi, 3)
    dx = data - dataSL
    
    # apply a little function to calculate r^-6 for each (dx, dy, dz)
    calc_rminus6 = lambda dx: (np.linalg.norm(dx)*1e-10)**-6
    rminus6 = np.apply_along_axis(calc_rminus6, 2, dx)
    
    # saving array of all distances (before averaging)
    rm6_filename = './saved_rm6_{}'.format(spinlabel)
    np.save(rm6_filename,rminus6)
   

    
    print("Averaging the ensemble and calculating PRE distances...")
    # finally take the ensemble average
    #rminus6_avg = np.mean(rminus6[0:10000,:],axis=0) # only average first 10000 steps
    rminus6_avg = np.mean(rminus6,axis=0)
    
    # calculating 'average' distances in Angstroms
    r_angstrom = (rminus6_avg**(-1/6))*1e10
    distance_filename = './saved_avg_dist_{}'.format(spinlabel)
    np.save(distance_filename,r_angstrom)
   

    
    print("Calculating PRE rates...")
    # finally take the ensemble average
    #rminus6_avg = np.mean(rminus6[0:10000,:],axis=0) # only average first 10000 steps
    #rminus6_avg = np.mean(rminus6,axis=0)
    rate = (K*rminus6_avg)*(4*tau_C+((3*tau_C)/(1+(larmor_H**2)*(tau_C**2))))
    rates_744 = rate
    rate_filename = "./saved_rate_{}".format(spinlabel)
    np.save(rate_filename,rates_744)
    
    print("Calculating intensity ratios...")
    ratios_744 = (R2H*np.exp(-2*DELTA*rate)/(R2H+rate))*(R2MQ/(R2MQ+rate))
    ratio_filename = './saved_ratio_{}'.format(spinlabel)
    np.save(ratio_filename,ratios_744)
 
                                                           
                                                           
print("Done!")  

In [ ]:
# load ratios for each probe site
ratios_657 = np.load('./saved_ratio_SC_657.npy')
ratios_699 = np.load('./saved_ratio_SC_699.npy')
ratios_744 = np.load('./saved_ratio_SC_744.npy')

In [ ]:
# add 0 values into the ratios list for probe site residues
new_ratios_657 = np.insert(ratios_657,11,0)

new_ratios_699 = np.insert(ratios_699,53,0)

new_ratios_744 = np.insert(ratios_744,98,0)

# Plot PRE Profiles

### Just Ensemble Profiles

In [ ]:
fig1 = plt.figure(figsize =(12,8))
ax1 = fig1.add_subplot(3,1,1)
ax2 = fig1.add_subplot(3,1,2)
ax3 = fig1.add_subplot(3,1,3)

fig1.tight_layout()

ax3.set_xlabel('Residue')
ax1.set_ylabel('Iox/Ired')
ax2.set_ylabel('Iox/Ired')
ax3.set_ylabel('Iox/Ired')

ax1.set_title('E657')
ax2.set_title('E699')
ax3.set_title('E744')

residues_fln5 = list(range(646,751))

ax1.plot(residues_fln5,new_ratios_657,'-k')
ax2.plot(residues_fln5,new_ratios_699,'-k')
ax3.plot(residues_fln5,new_ratios_744,'-k')

plt.draw()

#fig1.savefig('./PRE_predict_a3a3_150.png')

### Experimental Profile vs Ensemble Profile

In [ ]:
# plotting predicted profiles (replica averaged) for positions with experimental data

# load experimental data
c657 = pd.read_csv('./a3a3_c657_ratios_new.csv')
c699 = pd.read_csv('./a3a3_c699_ratios_new.csv')
c744 = pd.read_csv('./a3a3_c744_ratios_new.csv')

fig2, ax = plt.subplots(nrows=3,ncols=1,figsize=(15, 20))
fig2.subplots_adjust(hspace=.5,wspace=0.4)

residues_fln5 = list(range(646,751))

# plotting E657
plt.subplot(3,1,1)
plt.title('E657')
plt.xlabel("Residue")
plt.ylabel("Iox/Ired")


plt.plot(residues_fln5,new_ratios_657,'-k',linewidth = 2,label='160K isolated A3A3 FLN5 Ensemble Prediction')
plt.bar(c657['Residue'],c657['Ratio_paramagnetic:diamagnetic'],color = 'cyan',yerr = c657['Combined_error'],
       edgecolor = 'black',linewidth = 0.5,error_kw=dict(ecolor='black',elinewidth=0.5)
       ,label = 'isolated A3A3 Exp. Data')
plt.legend(loc = 'best')

# plotting D699
plt.subplot(3,1,2)
plt.title('E699')
plt.xlabel("Residue")
plt.ylabel("Iox/Ired")


plt.plot(residues_fln5,new_ratios_699,'-k',linewidth = 2,label='160K isolated A3A3 FLN5 Ensemble Prediction')
plt.bar(c699['Residue'],c699['Ratio_paramagnetic:diamagnetic'],color = 'cyan',yerr = c699['Combined_error'],
       edgecolor = 'black',linewidth = 0.5,error_kw=dict(ecolor='black',elinewidth=0.5)
       ,label = '+isolated A3A3 Exp. Data')
plt.legend(loc = 'best')

# plotting D744
plt.subplot(3,1,3)
plt.title('E744')
plt.xlabel("Residue")
plt.ylabel("Iox/Ired")



plt.plot(residues_fln5,new_ratios_744,'-k',linewidth = 2,label='160K isolated A3A3 FLN5 Ensemble Prediction')
plt.bar(c744['Residue'],c744['Ratio_paramagnetic:diamagnetic'],color = 'cyan',yerr = c744['Combined_error'],
       edgecolor = 'black',linewidth = 0.5,error_kw=dict(ecolor='black',elinewidth=0.5)
       ,label = 'isolated A3A3 Exp. Data')

plt.legend(loc = 'best')

plt.draw()
#plt.savefig('./exp_predict_compare.png')